## Tools

In [ ]:
!pip install datasets GPUtil bitsandbytes torch-pruning

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.1/64.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import login

login(token="HUGGING_FACE_KEY")

In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding
import time
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, classification_report
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

from datasets import load_dataset
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DistilBertForSequenceClassification,
    Trainer,
    TrainingArguments,
    TrainerCallback
)
import numpy as np
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    confusion_matrix,
    roc_curve,
    auc,
    f1_score
)
import psutil
import GPUtil
import pandas as pd

## Network Intrusion Dataset

In [ ]:
from datasets import load_dataset

dataset = load_dataset("gates04/network-intrusion-dataset")

train_dataset = dataset["train"]
validation_dataset = dataset["validation"]
test_dataset = dataset["test"]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/586 [00:00<?, ?B/s]

train-00000-of-00005.parquet:   0%|          | 0.00/135M [00:00<?, ?B/s]

train-00001-of-00005.parquet:   0%|          | 0.00/135M [00:00<?, ?B/s]

train-00002-of-00005.parquet:   0%|          | 0.00/135M [00:00<?, ?B/s]

...

validation-00000-of-00001.parquet:   0%|          | 0.00/145M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/145M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1187781 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/254525 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/254525 [00:00<?, ? examples/s]

## Unstructured Pruning NID Model

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
unstructured_pruning_model_name = "gates04/Unstructured-Pruning-NID"
tokenizer = AutoTokenizer.from_pretrained(unstructured_pruning_model_name)
unstructured_pruning_model = AutoModelForSequenceClassification.from_pretrained(unstructured_pruning_model_name)
unstructured_pruning_model.eval()

unstructured_pruning_model.to(device)

tokenizer_config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.81k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


## Structured Pruning NID Model

In [ ]:
structured_pruning_model_name = "gates04/Structured-Pruning-NID"
tokenizer = AutoTokenizer.from_pretrained(structured_pruning_model_name)
structured_pruning_model = AutoModelForSequenceClassification.from_pretrained(structured_pruning_model_name)
structured_pruning_model.eval()

structured_pruning_model.to(device)

tokenizer_config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.81k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


## FP16-KD-NID Model

In [ ]:
from transformers import AutoConfig

fp16_kd_model_name = "gates04/FP16-KD-NID"

labels = [
    'Analysis', 'Backdoor', 'Bot', 'DDoS', 'DoS', 'DoS GoldenEye',
    'DoS Hulk', 'DoS SlowHTTPTest', 'DoS Slowloris', 'Exploits',
    'FTP Patator', 'Fuzzers', 'Generic', 'Heartbleed', 'Infiltration',
    'Normal', 'Port Scan', 'Reconnaissance', 'SSH Patator',
    'Shellcode', 'Web Attack - Brute Force', 'Web Attack - SQL Injection',
    'Web Attack - XSS', 'Worms'
]

config = AutoConfig.from_pretrained(fp16_kd_model_name)

id2label = {str(i): label for i, label in enumerate(labels)}
label2id = {label: i for i, label in enumerate(labels)}

config.id2label = id2label
config.label2id = label2id

tokenizer = AutoTokenizer.from_pretrained(fp16_kd_model_name)
fp16_kd_model = AutoModelForSequenceClassification.from_pretrained(fp16_kd_model_name)
fp16_kd_model.eval()

fp16_kd_model.to(device)

config.json:   0%|          | 0.00/1.78k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/76.2M [00:00<?, ?B/s]

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 312, padding_idx=0)
      (position_embeddings): Embedding(512, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-3): 4 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=312, out_features=312, bias=True)
            (k_lin): Linear(in_features=312, out_features=312, bias=True)
            (v_lin): Linear(in_features=312, out_features=312, bias=True)
            (out_lin): Linear(in_features=312, out_features=312, bias=True)
          )
          (sa_layer_norm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


## NF4-KD-NID Model

In [ ]:
nf4_kd_model_name = "gates04/NF4-KD-NID"

labels = [
    'Analysis', 'Backdoor', 'Bot', 'DDoS', 'DoS', 'DoS GoldenEye',
    'DoS Hulk', 'DoS SlowHTTPTest', 'DoS Slowloris', 'Exploits',
    'FTP Patator', 'Fuzzers', 'Generic', 'Heartbleed', 'Infiltration',
    'Normal', 'Port Scan', 'Reconnaissance', 'SSH Patator',
    'Shellcode', 'Web Attack - Brute Force', 'Web Attack - SQL Injection',
    'Web Attack - XSS', 'Worms'
]

config = AutoConfig.from_pretrained(nf4_kd_model_name)

id2label = {str(i): label for i, label in enumerate(labels)}
label2id = {label: i for i, label in enumerate(labels)}

config.id2label = id2label
config.label2id = label2id

tokenizer = AutoTokenizer.from_pretrained(nf4_kd_model_name)
nf4_kd_model = AutoModelForSequenceClassification.from_pretrained(nf4_kd_model_name)
nf4_kd_model.eval()

nf4_kd_model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 312, padding_idx=0)
      (position_embeddings): Embedding(512, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-3): 4 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=312, out_features=312, bias=True)
            (k_lin): Linear(in_features=312, out_features=312, bias=True)
            (v_lin): Linear(in_features=312, out_features=312, bias=True)
            (out_lin): Linear(in_features=312, out_features=312, bias=True)
          )
          (sa_layer_norm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


## Structured Pruning + FP16 Quantization NID Model

In [ ]:
from transformers import BitsAndBytesConfig
import torch.nn.utils.prune as prune

model_name = "gates04/Structured-Pruning-NID"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

structured_fp16_model = model.half()

structured_fp16_model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
structured_fp16_model.to(device)

tokenizer_config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.81k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


## Structured Pruning + NF4 Quantization NID Model

In [ ]:
from transformers import BitsAndBytesConfig
import torch.nn.utils.prune as prune

model_name = "gates04/Structured-Pruning-NID"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)


bnb_config_int4 = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)


structured_nf4_model = AutoModelForSequenceClassification.from_pretrained(
    model_name, quantization_config=bnb_config_int4, device_map="cuda"
)


structured_nf4_model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
structured_nf4_model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear4bit(in_features=768, out_features=768, bias=True)
            (k_lin): Linear4bit(in_features=768, out_features=768, bias=True)
            (v_lin): Linear4bit(in_features=768, out_features=768, bias=True)
            (out_lin): Linear4bit(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1,

## Structured-FP16-KD NID Model

In [ ]:
from transformers import AutoConfig

structured_fp16_kd_model_name = "gates04/Structured-FP16-KD-NID"

labels = [
    'Analysis', 'Backdoor', 'Bot', 'DDoS', 'DoS', 'DoS GoldenEye',
    'DoS Hulk', 'DoS SlowHTTPTest', 'DoS Slowloris', 'Exploits',
    'FTP Patator', 'Fuzzers', 'Generic', 'Heartbleed', 'Infiltration',
    'Normal', 'Port Scan', 'Reconnaissance', 'SSH Patator',
    'Shellcode', 'Web Attack - Brute Force', 'Web Attack - SQL Injection',
    'Web Attack - XSS', 'Worms'
]

config = AutoConfig.from_pretrained(structured_fp16_kd_model_name)

id2label = {str(i): label for i, label in enumerate(labels)}
label2id = {label: i for i, label in enumerate(labels)}

config.id2label = id2label
config.label2id = label2id

tokenizer = AutoTokenizer.from_pretrained(structured_fp16_kd_model_name)
structured_fp16_kd_model = AutoModelForSequenceClassification.from_pretrained(structured_fp16_kd_model_name)
structured_fp16_kd_model.eval()

structured_fp16_kd_model.to(device)

config.json:   0%|          | 0.00/1.78k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/76.2M [00:00<?, ?B/s]

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 312, padding_idx=0)
      (position_embeddings): Embedding(512, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-3): 4 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=312, out_features=312, bias=True)
            (k_lin): Linear(in_features=312, out_features=312, bias=True)
            (v_lin): Linear(in_features=312, out_features=312, bias=True)
            (out_lin): Linear(in_features=312, out_features=312, bias=True)
          )
          (sa_layer_norm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


## Structured-NF4-KD NID Model

In [ ]:
from transformers import AutoConfig

structured_nf4_kd_model_name = "gates04/Structured-NF4-KD-NID"

labels = [
    'Analysis', 'Backdoor', 'Bot', 'DDoS', 'DoS', 'DoS GoldenEye',
    'DoS Hulk', 'DoS SlowHTTPTest', 'DoS Slowloris', 'Exploits',
    'FTP Patator', 'Fuzzers', 'Generic', 'Heartbleed', 'Infiltration',
    'Normal', 'Port Scan', 'Reconnaissance', 'SSH Patator',
    'Shellcode', 'Web Attack - Brute Force', 'Web Attack - SQL Injection',
    'Web Attack - XSS', 'Worms'
]

config = AutoConfig.from_pretrained(structured_nf4_kd_model_name)

id2label = {str(i): label for i, label in enumerate(labels)}
label2id = {label: i for i, label in enumerate(labels)}

config.id2label = id2label
config.label2id = label2id

tokenizer = AutoTokenizer.from_pretrained(structured_nf4_kd_model_name)
structured_nf4_kd_model = AutoModelForSequenceClassification.from_pretrained(structured_nf4_kd_model_name)
structured_nf4_kd_model.eval()

structured_nf4_kd_model.to(device)

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/76.2M [00:00<?, ?B/s]

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 312, padding_idx=0)
      (position_embeddings): Embedding(512, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-3): 4 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=312, out_features=312, bias=True)
            (k_lin): Linear(in_features=312, out_features=312, bias=True)
            (v_lin): Linear(in_features=312, out_features=312, bias=True)
            (out_lin): Linear(in_features=312, out_features=312, bias=True)
          )
          (sa_layer_norm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


## NF4-KD-SD-NID Model

In [ ]:
from transformers import AutoConfig

fp16_kd_sd_model_name = "gates04/NF4-KD-SD-NID"

labels = [
    'Analysis', 'Backdoor', 'Bot', 'DDoS', 'DoS', 'DoS GoldenEye',
    'DoS Hulk', 'DoS SlowHTTPTest', 'DoS Slowloris', 'Exploits',
    'FTP Patator', 'Fuzzers', 'Generic', 'Heartbleed', 'Infiltration',
    'Normal', 'Port Scan', 'Reconnaissance', 'SSH Patator',
    'Shellcode', 'Web Attack - Brute Force', 'Web Attack - SQL Injection',
    'Web Attack - XSS', 'Worms'
]

config = AutoConfig.from_pretrained(fp16_kd_sd_model_name)

id2label = {str(i): label for i, label in enumerate(labels)}
label2id = {label: i for i, label in enumerate(labels)}

config.id2label = id2label
config.label2id = label2id

tokenizer = AutoTokenizer.from_pretrained(fp16_kd_sd_model_name)
fp16_kd_sd_model = AutoModelForSequenceClassification.from_pretrained(fp16_kd_sd_model_name)
fp16_kd_sd_model.eval()

fp16_kd_sd_model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 312, padding_idx=0)
      (position_embeddings): Embedding(512, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-3): 4 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=312, out_features=312, bias=True)
            (k_lin): Linear(in_features=312, out_features=312, bias=True)
            (v_lin): Linear(in_features=312, out_features=312, bias=True)
            (out_lin): Linear(in_features=312, out_features=312, bias=True)
          )
          (sa_layer_norm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


## Structured-FP16-KD-SD NID Model

In [ ]:
from transformers import AutoConfig

structured_fp16_kd_sd_model_name = "gates04/Structured-FP16-KD-SD-NID"

labels = [
    'Analysis', 'Backdoor', 'Bot', 'DDoS', 'DoS', 'DoS GoldenEye',
    'DoS Hulk', 'DoS SlowHTTPTest', 'DoS Slowloris', 'Exploits',
    'FTP Patator', 'Fuzzers', 'Generic', 'Heartbleed', 'Infiltration',
    'Normal', 'Port Scan', 'Reconnaissance', 'SSH Patator',
    'Shellcode', 'Web Attack - Brute Force', 'Web Attack - SQL Injection',
    'Web Attack - XSS', 'Worms'
]

config = AutoConfig.from_pretrained(structured_fp16_kd_sd_model_name)

id2label = {str(i): label for i, label in enumerate(labels)}
label2id = {label: i for i, label in enumerate(labels)}

config.id2label = id2label
config.label2id = label2id

tokenizer = AutoTokenizer.from_pretrained(structured_fp16_kd_sd_model_name)
structured_fp16_kd_sd_model = AutoModelForSequenceClassification.from_pretrained(structured_fp16_kd_sd_model_name)
structured_fp16_kd_sd_model.eval()

structured_fp16_kd_sd_model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 312, padding_idx=0)
      (position_embeddings): Embedding(512, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-3): 4 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=312, out_features=312, bias=True)
            (k_lin): Linear(in_features=312, out_features=312, bias=True)
            (v_lin): Linear(in_features=312, out_features=312, bias=True)
            (out_lin): Linear(in_features=312, out_features=312, bias=True)
          )
          (sa_layer_norm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


## Structured-NF4-KD-SD NID Model

In [ ]:
from transformers import AutoConfig

structured_nf4_kd_sd_model_name = "gates04/Structured-NF4-KD-SD-NID"

labels = [
    'Analysis', 'Backdoor', 'Bot', 'DDoS', 'DoS', 'DoS GoldenEye',
    'DoS Hulk', 'DoS SlowHTTPTest', 'DoS Slowloris', 'Exploits',
    'FTP Patator', 'Fuzzers', 'Generic', 'Heartbleed', 'Infiltration',
    'Normal', 'Port Scan', 'Reconnaissance', 'SSH Patator',
    'Shellcode', 'Web Attack - Brute Force', 'Web Attack - SQL Injection',
    'Web Attack - XSS', 'Worms'
]

config = AutoConfig.from_pretrained(structured_nf4_kd_sd_model_name)

id2label = {str(i): label for i, label in enumerate(labels)}
label2id = {label: i for i, label in enumerate(labels)}

config.id2label = id2label
config.label2id = label2id

tokenizer = AutoTokenizer.from_pretrained(structured_nf4_kd_sd_model_name)
structured_nf4_kd_sd_model = AutoModelForSequenceClassification.from_pretrained(structured_nf4_kd_sd_model_name)
structured_nf4_kd_sd_model.eval()


structured_nf4_kd_sd_model.to(device)

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

KeyboardInterrupt: 

## Tokenization

In [ ]:
from transformers import AutoConfig
# Modelin config'inden label mapping'i al (eğer varsa)
#config = AutoConfig.from_pretrained(model_name)
if hasattr(config, "label2id"):
    label_mapping = config.label2id  # Örneğin: {"DoS": 0, "Normal": 1, ...}
    reverse_label_mapping = {v: k for k, v in label_mapping.items()}
else:
    # Eğer config'de mapping yoksa, manuel oluşturmak gerekir.
    # Örneğin, bunu modelin eğitildiği veriden öğrenmelisiniz.
    label_mapping = {"Normal": 0, "DoS": 1, "Probe": 2, "U2R": 3, "R2L": 4}  # Örnek!
    reverse_label_mapping = {v: k for k, v in label_mapping.items()}

print("Label Mapping:", label_mapping)

Label Mapping: {'Analysis': 0, 'Backdoor': 1, 'Bot': 2, 'DDoS': 3, 'DoS': 4, 'DoS GoldenEye': 5, 'DoS Hulk': 6, 'DoS SlowHTTPTest': 7, 'DoS Slowloris': 8, 'Exploits': 9, 'FTP Patator': 10, 'Fuzzers': 11, 'Generic': 12, 'Heartbleed': 13, 'Infiltration': 14, 'Normal': 15, 'Port Scan': 16, 'Reconnaissance': 17, 'SSH Patator': 18, 'Shellcode': 19, 'Web Attack - Brute Force': 20, 'Web Attack - SQL Injection': 21, 'Web Attack - XSS': 22, 'Worms': 23}


In [ ]:
max_length = 512

def tokenize_dataset(examples):
    texts = examples["packet_dat"]
    tokenized_inputs = tokenizer(texts, padding="max_length", truncation=True, max_length=max_length)
    #tokenized_inputs["labels"] = label_mapping[examples["attack_cat"]]
    tokenized_inputs["labels"] = [label_mapping[label] for label in examples["attack_cat"]]
    return tokenized_inputs


train_encoded = train_dataset.map(tokenize_dataset, batched=True, num_proc=16)
val_encoded   = validation_dataset.map(tokenize_dataset, batched=True, num_proc=16)
test_encoded  = test_dataset.map(tokenize_dataset, batched=True, num_proc=16)

train_encoded.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
val_encoded.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
test_encoded.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

train_encoded = train_encoded.remove_columns(["packet_dat", "attack_cat"])
val_encoded = val_encoded.remove_columns(["packet_dat", "attack_cat"])
test_encoded = test_encoded.remove_columns(["packet_dat", "attack_cat"])

Map (num_proc=16):   0%|          | 0/1187781 [00:00<?, ? examples/s]

Map (num_proc=16):   0%|          | 0/254525 [00:00<?, ? examples/s]

Map (num_proc=16):   0%|          | 0/254525 [00:00<?, ? examples/s]

## Model Size

In [ ]:
from transformers import BitsAndBytesConfig

nf4_kd_fp16_model = nf4_kd_model.half()
nf4_kd_fp16_model.to(device)

nf4_kd_fp16_model.eval()


bnb_config_nf4 = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)


nf4_kd_nf4_model = AutoModelForSequenceClassification.from_pretrained(
    nf4_kd_model_name, quantization_config=bnb_config_nf4, device_map="cuda"
)


nf4_kd_nf4_model.eval()

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 312, padding_idx=0)
      (position_embeddings): Embedding(512, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-3): 4 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear4bit(in_features=312, out_features=312, bias=True)
            (k_lin): Linear4bit(in_features=312, out_features=312, bias=True)
            (v_lin): Linear4bit(in_features=312, out_features=312, bias=True)
            (out_lin): Linear4bit(in_features=312, out_features=312, bias=True)
          )
          (sa_layer_norm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1,

In [ ]:
dtype_of_first_param = next(unstructured_pruning_model.parameters()).dtype
print("Unstructured Pruning NID Model dtype:", dtype_of_first_param)

dtype_of_first_param = next(structured_pruning_model.parameters()).dtype
print("Structured Pruning NID Model dtype:", dtype_of_first_param)

Unstructured Pruning NID Model dtype: torch.float32
Structured Pruning NID Model dtype: torch.float32


In [ ]:
dtype_of_first_param = next(nf4_kd_model.parameters()).dtype
print("NF4 Knowledge Distillation NID Model dtype:", dtype_of_first_param)

dtype_of_first_param = next(fp16_kd_model.parameters()).dtype
print("FP16 Knowledge Distillation NID Model dtype:", dtype_of_first_param)

NF4 Knowledge Distillation NID Model dtype: torch.float32
FP16 Knowledge Distillation NID Model dtype: torch.float32


In [ ]:
dtype_of_first_param = next(structured_nf4_model.parameters()).dtype
print("Structured Pruning + NF4 Quantization NID Model dtype:", dtype_of_first_param)

dtype_of_first_param = next(structured_fp16_model.parameters()).dtype
print("Structured Pruning + FP16 Quantization NID Model dtype:", dtype_of_first_param)

Structured Pruning + NF4 Quantization NID Model dtype: torch.float16
Structured Pruning + FP16 Quantization NID Model dtype: torch.float16


In [ ]:
dtype_of_first_param = next(nf4_kd_fp16_model.parameters()).dtype
print("NF4 Knowledge Distillation NID (FP16) Model dtype:", dtype_of_first_param)

dtype_of_first_param = next(nf4_kd_nf4_model.parameters()).dtype
print("FP16 Knowledge Distillation NID (NF4) Model dtype:", dtype_of_first_param)

NF4 Knowledge Distillation NID (FP16) Model dtype: torch.float16
FP16 Knowledge Distillation NID (NF4) Model dtype: torch.float16


In [ ]:
def get_model_size(model, dtype):
    num_params = sum(p.numel() for p in model.parameters())  # Toplam parametre sayısı
    bits_per_param = 32 if dtype == torch.float32 else 16 if dtype == torch.float16 else 8 if dtype == "int8" else 4
    size_mb = (num_params * bits_per_param) / (8 * 1024 * 1024)  # MB cinsinden hesapla
    return num_params, size_mb

num_params_up, size_up = get_model_size(structured_nf4_model, torch.float16)
num_params_sp, size_sp = get_model_size(structured_fp16_model, torch.float16)

print(f"Structured Pruning - NF4 NID Model:  Parametre Sayısı: {num_params_up:,}  |  Boyut: {size_up:.2f} MB")
print(f"Structured Pruning - FP16 NID Model:  Parametre Sayısı: {num_params_sp:,}  |  Boyut: {size_sp:.2f} MB")

Structured Pruning - NF4 NID Model:  Parametre Sayısı: 44,271,384  |  Boyut: 84.44 MB
Structured Pruning - FP16 NID Model:  Parametre Sayısı: 65,799,960  |  Boyut: 125.50 MB


## Model Evaluation

In [ ]:
def evaluate_model(model, test_data, tokenizer, device, batch_size=16, num_workers=16):
    all_preds = []
    all_labels = []
    inference_times = []
    latencies = []
    gpu_mem_usage = []
    cpu_mem_usage = []
    avg_gpu_memory = 0
    avg_cpu_memory = 0
    throughput = 0
    class_report = None


    model.eval()

    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
    test_dataloader = DataLoader(test_data, batch_size=batch_size, collate_fn=collate_fn, num_workers=num_workers)

    total_batches = len(test_dataloader)

    with torch.inference_mode():
        for batch_idx, batch in enumerate(test_dataloader):
            input_ids = batch["input_ids"].to(device, non_blocking=True)
            attention_mask = batch["attention_mask"].to(device, non_blocking=True)
            labels = batch.get("labels", None)

            if torch.cuda.is_available():

                gpu_memory_peak = torch.cuda.max_memory_allocated(device) / 1024**2
                gpu_mem_usage.append(gpu_memory_peak)

            cpu_memory = psutil.Process().memory_info().rss / 1024**2
            cpu_mem_usage.append(cpu_memory)

            start_time = time.perf_counter()
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            inference_time = time.perf_counter() - start_time

            preds = torch.argmax(outputs.logits, dim=-1).cpu().numpy()
            all_preds.append(preds)
            inference_times.append(inference_time)
            latencies.append(inference_time / batch_size)

            if labels is not None:
                all_labels.append(labels.cpu().numpy())


            print(f"Processing batch {batch_idx+1}/{total_batches} - Remaining: {total_batches - (batch_idx + 1)} batches")

    all_preds = np.concatenate(all_preds)

    if all_labels:
        all_labels = np.concatenate(all_labels)
        accuracy = accuracy_score(all_labels, all_preds)
        precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average="macro")
        cm = confusion_matrix(all_labels, all_preds)

        target_names = [
            "Analysis", "Backdoor", "Bot", "DDoS", "DoS", "DoS GoldenEye", "DoS Hulk", "DoS SlowHTTPTest",
            "DoS Slowloris", "Exploits", "FTP Patator", "Fuzzers", "Generic", "Heartbleed", "Infiltration",
            "Normal", "Port Scan", "Reconnaissance", "SSH Patator", "Shellcode",
            "Web Attack - Brute Force", "Web Attack - SQL Injection", "Web Attack - XSS", "Worms"
        ]

        class_report = classification_report(all_labels, all_preds, target_names=target_names, digits=4, output_dict=True)
    else:
        accuracy, precision, recall, f1, cm = None, None, None, None, None


    total_inference_time = sum(inference_times)
    total_samples = len(test_data)

    if total_inference_time > 0:
      throughput = total_samples / total_inference_time
    else:
      throughput = 0


    avg_gpu_memory = np.mean(gpu_mem_usage) if len(gpu_mem_usage) > 0 else 0
    avg_cpu_memory = np.mean(cpu_mem_usage) if len(cpu_mem_usage) > 0 else 0

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "avg_inference_time": np.mean(inference_times),
        "inference_latency": np.mean(latencies),
        "confusion_matrix": cm,
        "throughput" : throughput,
        "avg_gpu_memory_usage": avg_gpu_memory,
        "avg_cpu_memory_usage": avg_cpu_memory,
        "classification_report": class_report

    }


def collate_fn(batch):
    batch_dict = {key: [d[key] for d in batch] for key in batch[0]}

    batch_dict["labels"] = torch.tensor(batch_dict["labels"], dtype=torch.long)

    batch_dict["input_ids"] = torch.nn.utils.rnn.pad_sequence(
        [torch.tensor(ids, dtype=torch.long) for ids in batch_dict["input_ids"]],
        batch_first=True, padding_value=0
    )
    batch_dict["attention_mask"] = torch.nn.utils.rnn.pad_sequence(
        [torch.tensor(mask, dtype=torch.long) for mask in batch_dict["attention_mask"]],
        batch_first=True, padding_value=0
    )

    return batch_dict


def get_model_size(model):
    param_size = sum(p.numel() * p.element_size() for p in model.parameters())
    buffer_size = sum(b.numel() * b.element_size() for b in model.buffers())
    total_size = param_size + buffer_size
    return total_size / (1024**2)

## Model Inference

In [ ]:
from transformers import BitsAndBytesConfig

nf4_kd_fp16_model = nf4_kd_model.half()
nf4_kd_fp16_model.to(device)

nf4_kd_fp16_model.eval()

nf4_kd_nf4_model = nf4_kd_model.half()

bnb_config_int4 = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)


nf4_kd_nf4_model = AutoModelForSequenceClassification.from_pretrained(
    nf4_kd_model_name, quantization_config=bnb_config_int4, device_map="cuda"
)


nf4_kd_fp16_model.eval()

In [ ]:
print("\n Model (FP32) Evaluation on Test Set...")
results_model_up_test = evaluate_model(structured_fp16_kd_sd_model, test_encoded, tokenizer, device)

print("\n Model (FP16) Evaluation on Test Set...")
results_model_sp_test = evaluate_model(structured_nf4_kd_sd_model, test_encoded, tokenizer, device)


# Sonuçları gösterme
import pandas as pd


df_results = pd.DataFrame([
    {"Model": "Structured-FP16-KD-SD NID Model ", "Accuracy": results_model_up_test["accuracy"], "Precision": results_model_up_test["precision"], "Recall": results_model_up_test["recall"], "F1 Score": results_model_up_test["f1_score"], "Average Inference Time (s)": results_model_up_test["avg_inference_time"], "Inference Latency": results_model_up_test["inference_latency"], "Throughput": results_model_up_test["throughput"]},
    {"Model": "Structured-NF4-KD-SD NID Model ", "Accuracy": results_model_sp_test["accuracy"], "Precision": results_model_sp_test["precision"], "Recall": results_model_sp_test["recall"], "F1 Score": results_model_sp_test["f1_score"], "Average Inference Time (s)": results_model_sp_test["avg_inference_time"], "Inference Latency": results_model_sp_test["inference_latency"], "Throughput": results_model_sp_test["throughput"]},
])

print(df_results)


 Model (FP32) Evaluation on Test Set...


<ipython-input-14-42452cfb8040>:91: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [torch.tensor(ids, dtype=torch.long) for ids in batch_dict["input_ids"]],
<ipython-input-14-42452cfb8040>:91: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [torch.tensor(ids, dtype=torch.long) for ids in batch_dict["input_ids"]],
<ipython-input-14-42452cfb8040>:91: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [torch.tensor(ids, dtype=torch.long) for ids in batch_dict["input_ids"]],
<ipython-input-14-42452cfb8040>:91: UserWarning: To copy construct from a te

Streaming output truncated to the last 5000 lines.
Processing batch 10911/15908 - Remaining: 4997 batches
Processing batch 10912/15908 - Remaining: 4996 batches
Processing batch 10913/15908 - Remaining: 4995 batches
Processing batch 10914/15908 - Remaining: 4994 batches
Processing batch 10915/15908 - Remaining: 4993 batches
Processing batch 10916/15908 - Remaining: 4992 batches
Processing batch 10917/15908 - Remaining: 4991 batches
Processing batch 10918/15908 - Remaining: 4990 batches
Processing batch 10919/15908 - Remaining: 4989 batches
Processing batch 10920/15908 - Remaining: 4988 batches
Processing batch 10921/15908 - Remaining: 4987 batches
Processing batch 10922/15908 - Remaining: 4986 batches
Processing batch 10923/15908 - Remaining: 4985 batches
Processing batch 10924/15908 - Remaining: 4984 batches
Processing batch 10925/15908 - Remaining: 4983 batches
Processing batch 10926/15908 - Remaining: 4982 batches
Processing batch 10927/15908 - Remaining: 4981 batches
Processing bat

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
<ipython-input-14-42452cfb8040>:91: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [torch.tensor(ids, dtype=torch.long) for ids in batch_dict["input_ids"]],
<ipython-input-14-42452cfb8040>:91: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [

Streaming output truncated to the last 5000 lines.
Processing batch 10916/15908 - Remaining: 4992 batches
Processing batch 10917/15908 - Remaining: 4991 batches
Processing batch 10918/15908 - Remaining: 4990 batches
Processing batch 10919/15908 - Remaining: 4989 batches
Processing batch 10920/15908 - Remaining: 4988 batches
Processing batch 10921/15908 - Remaining: 4987 batches
Processing batch 10922/15908 - Remaining: 4986 batches
Processing batch 10923/15908 - Remaining: 4985 batches
Processing batch 10924/15908 - Remaining: 4984 batches
Processing batch 10925/15908 - Remaining: 4983 batches
Processing batch 10926/15908 - Remaining: 4982 batches
Processing batch 10927/15908 - Remaining: 4981 batches
Processing batch 10928/15908 - Remaining: 4980 batches
Processing batch 10929/15908 - Remaining: 4979 batches
Processing batch 10930/15908 - Remaining: 4978 batches
Processing batch 10931/15908 - Remaining: 4977 batches
Processing batch 10932/15908 - Remaining: 4976 batches
Processing bat

In [ ]:
print("\n Model (FP32) Evaluation on Test Set...")
results_model_up_test = evaluate_model(structured_fp16_kd_sd_model, test_encoded, tokenizer, device)

import pandas as pd

df_results = pd.DataFrame([
    {"Model": "Structured-FP16-KD-SD NID Model ", "Accuracy": results_model_up_test["accuracy"], "Precision": results_model_up_test["precision"], "Recall": results_model_up_test["recall"], "F1 Score": results_model_up_test["f1_score"], "Average Inference Time (s)": results_model_up_test["avg_inference_time"], "Inference Latency": results_model_up_test["inference_latency"], "Throughput": results_model_up_test["throughput"]},
])

print(df_results)

from pprint import pprint
pprint(results_model_up_test["classification_report"])


 Model (FP32) Evaluation on Test Set...


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
<ipython-input-13-1efa080aa96c>:106: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [torch.tensor(ids, dtype=torch.long) for ids in batch_dict["input_ids"]],
<ipython-input-13-1efa080aa96c>:106: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
 

Streaming output truncated to the last 5000 lines.
Processing batch 11019/15908 - Remaining: 4889 batches
Processing batch 11020/15908 - Remaining: 4888 batches
Processing batch 11021/15908 - Remaining: 4887 batches
Processing batch 11022/15908 - Remaining: 4886 batches
Processing batch 11023/15908 - Remaining: 4885 batches
Processing batch 11024/15908 - Remaining: 4884 batches
Processing batch 11025/15908 - Remaining: 4883 batches
Processing batch 11026/15908 - Remaining: 4882 batches
Processing batch 11027/15908 - Remaining: 4881 batches
Processing batch 11028/15908 - Remaining: 4880 batches
Processing batch 11029/15908 - Remaining: 4879 batches
Processing batch 11030/15908 - Remaining: 4878 batches
Processing batch 11031/15908 - Remaining: 4877 batches
Processing batch 11032/15908 - Remaining: 4876 batches
Processing batch 11033/15908 - Remaining: 4875 batches
Processing batch 11034/15908 - Remaining: 4874 batches
Processing batch 11035/15908 - Remaining: 4873 batches
Processing bat

In [ ]:
print("\n Model (FP32) Evaluation on Test Set...")
results_model_up_test = evaluate_model(structured_fp16_kd_sd_model, test_encoded, tokenizer, device)

import pandas as pd

df_results = pd.DataFrame([
    {"Model": "Structured-FP16-KD-SD NID Model ", "Accuracy": results_model_up_test["accuracy"], "Precision": results_model_up_test["precision"], "Recall": results_model_up_test["recall"], "F1 Score": results_model_up_test["f1_score"], "Average Inference Time (s)": results_model_up_test["avg_inference_time"], "Inference Latency": results_model_up_test["inference_latency"], "Throughput": results_model_up_test["throughput"]},
])

print(df_results)

from pprint import pprint
pprint(results["classification_report"])


 Model (FP32) Evaluation on Test Set...


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
<ipython-input-10-42452cfb8040>:91: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [torch.tensor(ids, dtype=torch.long) for ids in batch_dict["input_ids"]],
<ipython-input-10-42452cfb8040>:91: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [

Streaming output truncated to the last 5000 lines.
Processing batch 10914/15908 - Remaining: 4994 batches
Processing batch 10915/15908 - Remaining: 4993 batches
Processing batch 10916/15908 - Remaining: 4992 batches
Processing batch 10917/15908 - Remaining: 4991 batches
Processing batch 10918/15908 - Remaining: 4990 batches
Processing batch 10919/15908 - Remaining: 4989 batches
Processing batch 10920/15908 - Remaining: 4988 batches
Processing batch 10921/15908 - Remaining: 4987 batches
Processing batch 10922/15908 - Remaining: 4986 batches
Processing batch 10923/15908 - Remaining: 4985 batches
Processing batch 10924/15908 - Remaining: 4984 batches
Processing batch 10925/15908 - Remaining: 4983 batches
Processing batch 10926/15908 - Remaining: 4982 batches
Processing batch 10927/15908 - Remaining: 4981 batches
Processing batch 10928/15908 - Remaining: 4980 batches
Processing batch 10929/15908 - Remaining: 4979 batches
Processing batch 10930/15908 - Remaining: 4978 batches
Processing bat

In [ ]:
print("\n Model (FP32) Evaluation on Test Set...")
results_model_up_test = evaluate_model(structured_fp16_kd_model, test_encoded, tokenizer, device)

print("\n Model (FP16) Evaluation on Test Set...")
results_model_sp_test = evaluate_model(structured_nf4_kd_model, test_encoded, tokenizer, device)

import pandas as pd

df_results = pd.DataFrame([
    {"Model": "Structured-FP16-KD NID Model ", "Accuracy": results_model_up_test["accuracy"], "Precision": results_model_up_test["precision"], "Recall": results_model_up_test["recall"], "F1 Score": results_model_up_test["f1_score"], "Average Inference Time (s)": results_model_up_test["avg_inference_time"], "Inference Latency": results_model_up_test["inference_latency"], "Throughput": results_model_up_test["throughput"]},
    {"Model": "Structured-NF4-KD NID Model ", "Accuracy": results_model_sp_test["accuracy"], "Precision": results_model_sp_test["precision"], "Recall": results_model_sp_test["recall"], "F1 Score": results_model_sp_test["f1_score"], "Average Inference Time (s)": results_model_sp_test["avg_inference_time"], "Inference Latency": results_model_sp_test["inference_latency"], "Throughput": results_model_sp_test["throughput"]},
])

print(df_results)


 Model (FP32) Evaluation on Test Set...


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
<ipython-input-17-42452cfb8040>:91: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [torch.tensor(ids, dtype=torch.long) for ids in batch_dict["input_ids"]],
<ipython-input-17-42452cfb8040>:91: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [

Streaming output truncated to the last 5000 lines.
Processing batch 10911/15908 - Remaining: 4997 batches
Processing batch 10912/15908 - Remaining: 4996 batches
Processing batch 10913/15908 - Remaining: 4995 batches
Processing batch 10914/15908 - Remaining: 4994 batches
Processing batch 10915/15908 - Remaining: 4993 batches
Processing batch 10916/15908 - Remaining: 4992 batches
Processing batch 10917/15908 - Remaining: 4991 batches
Processing batch 10918/15908 - Remaining: 4990 batches
Processing batch 10919/15908 - Remaining: 4989 batches
Processing batch 10920/15908 - Remaining: 4988 batches
Processing batch 10921/15908 - Remaining: 4987 batches
Processing batch 10922/15908 - Remaining: 4986 batches
Processing batch 10923/15908 - Remaining: 4985 batches
Processing batch 10924/15908 - Remaining: 4984 batches
Processing batch 10925/15908 - Remaining: 4983 batches
Processing batch 10926/15908 - Remaining: 4982 batches
Processing batch 10927/15908 - Remaining: 4981 batches
Processing bat

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
<ipython-input-17-42452cfb8040>:91: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [torch.tensor(ids, dtype=torch.long) for ids in batch_dict["input_ids"]],
<ipython-input-17-42452cfb8040>:91: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [

Streaming output truncated to the last 5000 lines.
Processing batch 10916/15908 - Remaining: 4992 batches
Processing batch 10917/15908 - Remaining: 4991 batches
Processing batch 10918/15908 - Remaining: 4990 batches
Processing batch 10919/15908 - Remaining: 4989 batches
Processing batch 10920/15908 - Remaining: 4988 batches
Processing batch 10921/15908 - Remaining: 4987 batches
Processing batch 10922/15908 - Remaining: 4986 batches
Processing batch 10923/15908 - Remaining: 4985 batches
Processing batch 10924/15908 - Remaining: 4984 batches
Processing batch 10925/15908 - Remaining: 4983 batches
Processing batch 10926/15908 - Remaining: 4982 batches
Processing batch 10927/15908 - Remaining: 4981 batches
Processing batch 10928/15908 - Remaining: 4980 batches
Processing batch 10929/15908 - Remaining: 4979 batches
Processing batch 10930/15908 - Remaining: 4978 batches
Processing batch 10931/15908 - Remaining: 4977 batches
Processing batch 10932/15908 - Remaining: 4976 batches
Processing bat

In [ ]:
print("\n Model (FP32) Evaluation on Test Set...")
results_model_up_test = evaluate_model(fp16_kd_sd_model, test_encoded, tokenizer, device)

import pandas as pd

df_results = pd.DataFrame([
    {"Model": "FP16-KD-SD NID Model ", "Accuracy": results_model_up_test["accuracy"], "Precision": results_model_up_test["precision"], "Recall": results_model_up_test["recall"], "F1 Score": results_model_up_test["f1_score"], "Average Inference Time (s)": results_model_up_test["avg_inference_time"], "Inference Latency": results_model_up_test["inference_latency"], "Throughput": results_model_up_test["throughput"]},
])

print(df_results)


 Model (FP32) Evaluation on Test Set...


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
<ipython-input-12-42452cfb8040>:91: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [torch.tensor(ids, dtype=torch.long) for ids in batch_dict["input_ids"]],
<ipython-input-12-42452cfb8040>:91: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [

Streaming output truncated to the last 5000 lines.
Processing batch 10914/15908 - Remaining: 4994 batches
Processing batch 10915/15908 - Remaining: 4993 batches
Processing batch 10916/15908 - Remaining: 4992 batches
Processing batch 10917/15908 - Remaining: 4991 batches
Processing batch 10918/15908 - Remaining: 4990 batches
Processing batch 10919/15908 - Remaining: 4989 batches
Processing batch 10920/15908 - Remaining: 4988 batches
Processing batch 10921/15908 - Remaining: 4987 batches
Processing batch 10922/15908 - Remaining: 4986 batches
Processing batch 10923/15908 - Remaining: 4985 batches
Processing batch 10924/15908 - Remaining: 4984 batches
Processing batch 10925/15908 - Remaining: 4983 batches
Processing batch 10926/15908 - Remaining: 4982 batches
Processing batch 10927/15908 - Remaining: 4981 batches
Processing batch 10928/15908 - Remaining: 4980 batches
Processing batch 10929/15908 - Remaining: 4979 batches
Processing batch 10930/15908 - Remaining: 4978 batches
Processing bat

In [ ]:
print("\n Model (FP32) Evaluation on Test Set...")
results_model_up_test = evaluate_model(structured_nf4_model, test_encoded, tokenizer, device)

print("\n Model (FP16) Evaluation on Test Set...")
results_model_sp_test = evaluate_model(structured_fp16_model, test_encoded, tokenizer, device)

import pandas as pd

df_results = pd.DataFrame([
    {"Model": "Structured Pruning + NF4 Quantization NID Model ", "Accuracy": results_model_up_test["accuracy"], "Precision": results_model_up_test["precision"], "Recall": results_model_up_test["recall"], "F1 Score": results_model_up_test["f1_score"], "Average Inference Time (s)": results_model_up_test["avg_inference_time"], "Inference Latency": results_model_up_test["inference_latency"], "Throughput": results_model_up_test["throughput"]},
    {"Model": "Structured Pruning + FP16 Quantization NID Model ", "Accuracy": results_model_sp_test["accuracy"], "Precision": results_model_sp_test["precision"], "Recall": results_model_sp_test["recall"], "F1 Score": results_model_sp_test["f1_score"], "Average Inference Time (s)": results_model_sp_test["avg_inference_time"], "Inference Latency": results_model_sp_test["inference_latency"], "Throughput": results_model_sp_test["throughput"]},
])

print(df_results)


 Model (FP32) Evaluation on Test Set...


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
<ipython-input-14-42452cfb8040>:91: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [torch.tensor(ids, dtype=torch.long) for ids in batch_dict["input_ids"]],
<ipython-input-14-42452cfb8040>:91: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [

Streaming output truncated to the last 5000 lines.
Processing batch 10911/15908 - Remaining: 4997 batches
Processing batch 10912/15908 - Remaining: 4996 batches
Processing batch 10913/15908 - Remaining: 4995 batches
Processing batch 10914/15908 - Remaining: 4994 batches
Processing batch 10915/15908 - Remaining: 4993 batches
Processing batch 10916/15908 - Remaining: 4992 batches
Processing batch 10917/15908 - Remaining: 4991 batches
Processing batch 10918/15908 - Remaining: 4990 batches
Processing batch 10919/15908 - Remaining: 4989 batches
Processing batch 10920/15908 - Remaining: 4988 batches
Processing batch 10921/15908 - Remaining: 4987 batches
Processing batch 10922/15908 - Remaining: 4986 batches
Processing batch 10923/15908 - Remaining: 4985 batches
Processing batch 10924/15908 - Remaining: 4984 batches
Processing batch 10925/15908 - Remaining: 4983 batches
Processing batch 10926/15908 - Remaining: 4982 batches
Processing batch 10927/15908 - Remaining: 4981 batches
Processing bat

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
<ipython-input-14-42452cfb8040>:91: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [torch.tensor(ids, dtype=torch.long) for ids in batch_dict["input_ids"]],
<ipython-input-14-42452cfb8040>:91: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [

Streaming output truncated to the last 5000 lines.
Processing batch 10920/15908 - Remaining: 4988 batches
Processing batch 10921/15908 - Remaining: 4987 batches
Processing batch 10922/15908 - Remaining: 4986 batches
Processing batch 10923/15908 - Remaining: 4985 batches
Processing batch 10924/15908 - Remaining: 4984 batches
Processing batch 10925/15908 - Remaining: 4983 batches
Processing batch 10926/15908 - Remaining: 4982 batches
Processing batch 10927/15908 - Remaining: 4981 batches
Processing batch 10928/15908 - Remaining: 4980 batches
Processing batch 10929/15908 - Remaining: 4979 batches
Processing batch 10930/15908 - Remaining: 4978 batches
Processing batch 10931/15908 - Remaining: 4977 batches
Processing batch 10932/15908 - Remaining: 4976 batches
Processing batch 10933/15908 - Remaining: 4975 batches
Processing batch 10934/15908 - Remaining: 4974 batches
Processing batch 10935/15908 - Remaining: 4973 batches
Processing batch 10936/15908 - Remaining: 4972 batches
Processing bat